# About the dataset

<ul>
    <li>This dataset belongs to Trip Advisor hotels, where customer review texts and customer review ratings are collected </li>
    <li>20491 reviews with ratings</li>
    <li>Rating in 0 to 5</li>
    <li>Rating 0 to 1 is considered as bad (labeled as -1)</li>
    <li>Rating 2 to 3 is considered as neutral (labeled as 0)</li>
    <li>Rating 4 to 5 is considered as good (labeled as 1)</li>
</ul>

# Objective

<ul>
    <li>Train an NLP model with the given data </li>
    <li>Predict the rating based on unseen review text</li>
</ul>

# Acknowledgements

$\textbf{Citation: }$ Alam, M. H., Ryu, W.-J., Lee, S., 2016. Joint multi-grain topic sentiment: modeling semantic aspects for online reviews. Information Sciences 339, 206–223.

$\textbf{Link: }$ https://zenodo.org/records/1219899

# Necessary packages

In [1]:
import numpy as np
import pandas as pd